In [1]:
!pip install qutip

In [2]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

In [11]:
#This File contains all the parameters used in the program.
#Physical constants, transition rates, etc.

import numpy as np 

# Universal constants
pi = np.pi
h = 6.62607015e-34  # in J s
hbar = h/(2*pi)
c = 2.99792458e10   # cm/s
kB = 1.380649e-23   # J/K
eps0 = 8.8541878128e-14    # F/cm = C/ (V cm)
# Dipolar moment
d = 3.34e-31 # C cm

#Transition energies in J (we converted the data from cm^-1 to J by multiplying by hc)
w = 9811*h*c
w1 = 237*h*c
w2 = 138*h*c
w3 = 102*h*c
w4 = 132*h*c
w5 = 150*h*c
#Spontaneous Emission in s^-1
gamma = 314.5
#Non resonant transition rates in s^-1
gamma_nr = 1e12 
#Vibronic couplings in s^-1 (k/hbar)
k_12 = 6.35e11
k_23 = 2.8e11 
k_34 = 2.25e11
k_56 = 5.4e11 
k_67 = 4.1e11 

#Refraction index
n = 1.45
#Absortion Coefficients
alpha_imp = 4e-4 # cm-1
alpha_rad = 1e-2 # cm-1
#Energy levels
E0 = 0
E1 = w1
E2 = E1 + w2
E3 = E2 + w3
E4 = E3 + w
E5 = E4 + w4
E6 = E5 + w5 
E_gs = np.array([E0,E1,E2,E3])
E_es = np.array([E4,E5,E6])

H = np.zeros(7, dtype=np.complex128)
H[0] = E0
H[1] = E1
H[2] = E2
H[3] = E3
H[4] = E4
H[5] = E5
H[6] = E6



In [13]:
def Rabifreq(j_0):
    E_0 = np.sqrt(2*j_0/(c*n*eps0))
    return d*E_0/hbar

def N(beta,omega):
    return 1/(np.exp(beta*omega)-1)
def Thermal_state(beta, H):
    state = np.exp(-beta*H)
    state[-3:] = [0,0,0]
    return state / np.sum(state)





In [62]:
def Hamiltonian(T, j_0_3, j_0_4):
    beta = 1/(kB*T)
    Omega_4 = Rabifreq(j_0_4*1e6)
    Omega_3 = Rabifreq(j_0_3*1e6)
    ks = [k_12, k_23, k_34, Omega_4, k_56, k_67]
    ws = [w1, w2, w3, w, w4, w5]
    ###=========Hamiltonian=============
    H_ev = np.zeros((7,7), dtype = np.complex128)
    for i in range(6):
        H_ev[i, i+1] = ks[i]*(N(beta, ws[i])**0.5) 
    for i in range(1,7):
        H_ev[i, i-1] = ks[i-1]*(N(beta, ws[i-1])**0.5) 
    H_ev[3,4] = Omega_4
    H_ev[4,3] = Omega_4
    #H_ev[2,4] = Omega_3
    #H_ev[4,2] = Omega_3
    return H_ev 

In [87]:
c_nr = np.zeros((7,7))
for i in range(6):
    c_nr[i,i+1] = 1
c_nr[3,4] = 0
c_nr = np.sqrt(gamma_nr/1e9)* c_nr

c_nr = Qobj(c_nr)

c_se = np.zeros((7,7))
for i in range(4):
    for j in range(4, 7):
        c_se[i,j] = 1
c_se = np.sqrt(gamma/1e9)*c_nr
c_se = Qobj(c_se)


H = Hamiltonian(300, 0, 0)
H = Qobj(H/1e9) 
H

Quantum object: dims = [[7], [7]], shape = (7, 7), type = oper, isherm = True
Qobj data =
[[  0.         436.50493673   0.           0.           0.
    0.           0.        ]
 [436.50493673   0.         289.05166942   0.           0.
    0.           0.        ]
 [  0.         289.05166942   0.         283.25200587   0.
    0.           0.        ]
 [  0.           0.         283.25200587   0.           0.
    0.           0.        ]
 [  0.           0.           0.           0.           0.
  574.54341713   0.        ]
 [  0.           0.           0.           0.         574.54341713
    0.         399.51504246]
 [  0.           0.           0.           0.           0.
  399.51504246   0.        ]]

In [88]:
rho_ss = steadystate(H, [c_nr, c_nr.dag(), c_se])
rho_ss

Quantum object: dims = [[7], [7]], shape = (7, 7), type = oper, isherm = True
Qobj data =
[[2.50000067e-01+0.00000000e+00j 0.00000000e+00+4.65670868e-08j
  8.89183122e-09+0.00000000e+00j 0.00000000e+00+4.29352721e-10j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j]
 [0.00000000e+00-4.65670868e-08j 2.50000029e-01+0.00000000e+00j
  0.00000000e+00+5.07056833e-08j 4.78637482e-09+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j]
 [8.89183122e-09+0.00000000e+00j 0.00000000e+00-5.07056833e-08j
  2.49999979e-01+0.00000000e+00j 0.00000000e+00+4.31161822e-08j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j]
 [0.00000000e+00-4.29352721e-10j 4.78637482e-09+0.00000000e+00j
  0.00000000e+00-4.31161822e-08j 2.49999925e-01+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j]
 [0.00